# Count the signs

* ~~Write out the video output file. - this may need to be done inside the yolo file.~~
* Analyse the file, to get some idea of approach to counting.
* Extend the model so that it has an extra dictionary for the entry for the output so that it can overwrite the answer it provides.
* Then write out the end file so that 

In [1]:
#read the file in so that it pushes each found object into a dictionary with the frame as the key.
import os
base_dir = '/Users/datascience4/documents/yolo3/keras-yolo3/output/'

file = 'B4087_NB_RAV_R07_180519074106_completed_counting.txt'

file = open(os.path.join(base_dir, file), 'r')
file_read = file.readlines()
file.close()

In [2]:
print(file_read)

['Frame 1\n', '(416, 416, 3)\n', 'Found 0 boxes for  img\n', '1.9706918119991315\n', 'Frame 2\n', '(416, 416, 3)\n', 'Found 0 boxes for  img\n', '0.9735546739993879\n', 'Frame 3\n', '(416, 416, 3)\n', 'Found 0 boxes for  img\n', '0.9658381970002665\n', 'Frame 4\n', '(416, 416, 3)\n', 'Found 0 boxes for  img\n', '1.0914586590006365\n', 'Frame 5\n', '(416, 416, 3)\n', 'Found 0 boxes for  img\n', '0.981479070000205\n', 'Frame 6\n', '(416, 416, 3)\n', 'Found 0 boxes for  img\n', '1.067079668999213\n', 'Frame 7\n', '(416, 416, 3)\n', 'Found 0 boxes for  img\n', '1.0615692489991488\n', 'Frame 8\n', '(416, 416, 3)\n', 'Found 0 boxes for  img\n', '1.1094652119991224\n', 'Frame 9\n', '(416, 416, 3)\n', 'Found 0 boxes for  img\n', '1.0403507640003227\n', 'Frame 10\n', '(416, 416, 3)\n', 'Found 0 boxes for  img\n', '1.0496945030008646\n', 'Frame 11\n', '(416, 416, 3)\n', 'Found 0 boxes for  img\n', '1.0768533530008426\n', 'Frame 12\n', '(416, 416, 3)\n', 'Found 0 boxes for  img\n', '1.25548568500

In [171]:
import re
dic = {}
#leave the frame outside.
frame = {}
#objects in frame will need to be cleared inside
objects_in_frame = {}
for line in file_read:
    ## for frame, shape read
    if 'Frame' in line:
        new_line = list(line[:-1])
        frame_name = ''.join(str(letter) for letter in new_line)
        frame[frame_name] = {}
        if frame_name in line:
            line_count = file_read.index(line)
    if line == '(416, 416, 3)\n':
        line = list(line[:-1])
        frame[frame_name]['shape'] = ''.join(str(letter) for letter in line)
    if 'Found' in line:
        line = list(line)
        number_end = line.index('b') - 1
        number = line[6:number_end]
        frame[frame_name]['objects_count'] = int(''.join(str(n) for n in number))
        if frame[frame_name]['objects_count'] > 0:
            for i in range(frame[frame_name]['objects_count']):
                obj_line = (line_count + 2) + (i + 1)
                object_dic = frame[frame_name]['object ' + str(i)] = {}
                object_list = list(file_read[obj_line])

                obj_list = file_read[obj_line]
                sign_name = re.search('(\w+\s)+', obj_list)
                if sign_name != None:
                    sign_name = sign_name.group()
                    confidence_score = obj_list[len(sign_name):len(sign_name) + 4]
                    coord_pattern = r'\(([0-9])+\,\s([0-9])+\)'
                    left_top_coord = re.search(r'\(([0-9])+\,\s([0-9])+\)', obj_list).group()
                    right_bottom_coord = re.search(
                        r'(?<=\)\s)\(([0-9])+\,\s([0-9])+\)', obj_list).group()
                print(sign_name, len(sign_name))
                print(confidence_score)
                print(left_top_coord)
                print(right_bottom_coord)
                print(line_count, file_read[obj_line])                        
                    
#             frame[frame_name]['object ' + str(i+1)] = file_read.index(line)
            # add 1 to deal with the time taken to compute has been
            # entered below the frame fix later
            time_to_compute_line = line
            
#             frame[frame_name]['object ' + str(i)] = 


no entry sign  14
0.30
(1199, 39)
(1341, 178) <class 'str'>
112 no entry sign 0.30 (1199, 39) (1341, 178)

no entry sign  14
0.43
(549, 69)
(610, 152) <class 'str'>
125 no entry sign 0.43 (549, 69) (610, 152)

speed sign  11
0.43
(879, 60)
(922, 154) <class 'str'>
150 speed sign 0.43 (879, 60) (922, 154)

keep left sign  15
0.54
(920, 274)
(953, 321) <class 'str'>
263 keep left sign 0.54 (920, 274) (953, 321)

keep left sign  15
0.57
(643, 308)
(694, 379) <class 'str'>
268 keep left sign 0.57 (643, 308) (694, 379)

speed sign  11
0.41
(284, 139)
(319, 188) <class 'str'>
277 speed sign 0.41 (284, 139) (319, 188)

no entry sign  14
0.33
(185, 27)
(250, 101) <class 'str'>
382 no entry sign 0.33 (185, 27) (250, 101)

no entry sign  14
0.32
(1072, 18)
(1164, 190) <class 'str'>
387 no entry sign 0.32 (1072, 18) (1164, 190)

speed sign  11
0.37
(1142, 248)
(1163, 280) <class 'str'>
648 speed sign 0.37 (1142, 248) (1163, 280)

speed sign  11
0.31
(562, 200)
(579, 220) <class 'str'>
869 speed s

(50, 209) <class 'str'>
7867 speed sign 0.74 (9, 153) (50, 209)

keep left sign  15
0.55
(1207, 299)
(1266, 385) <class 'str'>
7873 keep left sign 0.55 (1207, 299) (1266, 385)

no entry sign  14
0.63
(187, 82)
(248, 180) <class 'str'>
7882 no entry sign 0.63 (187, 82) (248, 180)

speed sign  11
0.76
(1165, 161)
(1196, 208) <class 'str'>
7903 speed sign 0.76 (1165, 161) (1196, 208)

speed sign  11
0.86
(1173, 161)
(1208, 204) <class 'str'>
7908 speed sign 0.86 (1173, 161) (1208, 204)

speed sign  11
0.35
(620, 153)
(661, 201) <class 'str'>
7913 speed sign 0.35 (620, 153) (661, 201)

speed sign  11
0.68
(1195, 150)
(1256, 195) <class 'str'>
7913 speed sign 0.68 (1195, 150) (1256, 195)

speed sign  11
0.40
(564, 138)
(610, 182) <class 'str'>
7919 speed sign 0.40 (564, 138) (610, 182)

speed sign  11
0.46
(1241, 126)
(1313, 177) <class 'str'>
7919 speed sign 0.46 (1241, 126) (1313, 177)

speed sign  11
0.47
(1336, 109)
(1359, 164) <class 'str'>
7925 speed sign 0.47 (1336, 109) (1359, 164)


speed sign  11
0.37
(212, 103)
(267, 176) <class 'str'>
10730 speed sign 0.37 (212, 103) (267, 176)

speed sign  11
0.36
(879, 173)
(907, 211) <class 'str'>
11019 speed sign 0.36 (879, 173) (907, 211)

speed sign  11
0.35
(977, 149)
(1017, 205) <class 'str'>
11024 speed sign 0.35 (977, 149) (1017, 205)

no entry sign  14
0.71
(1158, 118)
(1221, 172) <class 'str'>
11029 no entry sign 0.71 (1158, 118) (1221, 172)

keep left sign  15
0.38
(851, 308)
(869, 332) <class 'str'>
11362 keep left sign 0.38 (851, 308) (869, 332)

no entry sign  14
0.30
(860, 175)
(880, 199) <class 'str'>
11483 no entry sign 0.30 (860, 175) (880, 199)

no entry sign  14
0.37
(1278, 129)
(1304, 157) <class 'str'>
11512 no entry sign 0.37 (1278, 129) (1304, 157)

speed sign  11
0.32
(1267, 248)
(1288, 288) <class 'str'>
11529 speed sign 0.32 (1267, 248) (1288, 288)

keep left sign  15
0.31
(1018, 333)
(1033, 358) <class 'str'>
11618 keep left sign 0.31 (1018, 333) (1033, 358)



In [158]:
frame['Frame 2']

{'shape': '(416, 416, 3)', 'objects_count': 0}

In [12]:
frame['Frame 1688']

{'shape': '(416, 416, 3)',
 'objects_count': 4,
 'object 0': {'hi': 'test'},
 'object 1': {'hi': 'test'},
 'object 2': {'hi': 'test'},
 'object 3': {'hi': 'test'}}

In [ ]:
print(line_count)